In [2]:
import os
from typing import List, Optional
import fire
from llama import Llama, Dialog
import torch

In [27]:
# Define major elements of Llama2 7b
os.environ['models_loc'] : str          =       'D:\GitHub repositories\llama\\'
os.environ['ckpt_dir']   : str          =       os.environ['models_loc'] + 'llama-2-7b-chat' 
os.environ['tokenizer_path'] : str      =       os.environ['models_loc'] + 'tokenizer.model'
os.environ['RANK']                      =       '0'
os.environ['WORLD_SIZE']                =       '1'
os.environ['MASTER_ADDR']               =       'localhost'
os.environ['MASTER_PORT']               =       '12355'
# Another type of initialization
# torch.distributed.init_process_group(
#     backend='nccl',
#     init_method='env://',
#     timeout=datetime.timedelta(0, 1800),
#     world_size=0,
#     rank=0,
#     store=None,
#     group_name=''
# )
#


In [28]:
# Main parameters of the model for further tuning
# for now, we leave some of them as default
temperature : float     =   0.6
top_p : float           =   0.9
max_seq_len : int       =   4096
max_batch_size : int    =   8
max_gen_len : Optional[int] =   None

In [29]:
# We are now Building a model, which takes a bit of time of course
generator = Llama.build(
    ckpt_dir=os.environ['ckpt_dir'],
    tokenizer_path=os.environ['tokenizer_path'],
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
)

> initializing model parallel with size 1


> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 70.35 seconds


In [30]:
# We need to make dialogs for the bot
Q1 : list[Dialog] = [

    #Q1
    
    [
        {
            "role"      :   "user",
            "content"   :   "At which geographical locations we could expect the presense of lithium for exploration and mining?"
        }
    ],
    [
            # From Wikipedia
        {   "role"      :   "system",
            "content"   :   """\
                Lithium is a chemical element with the symbol 'Li' and atomic number 3. It is a soft, silvery-white alkali metal.
                Under standard conditions, it is the least dense metal and the least dense solid element.
                Like all alkali metals, lithium is highly reactive and flammable, and must be stored in vacuum, inert atmosphere, or inert liquid such as purified kerosene or mineral oil.
                It exhibits a metallic luster. It corrodes quickly in air to a dull silvery gray, then black tarnish.
                It does not occur freely in nature, but occurs mainly as pegmatitic minerals, which were once the main source of lithium.
                Due to its solubility as an ion, it is present in ocean water and is commonly obtained from brines.
                Lithium metal is isolated electrolytically from a mixture of lithium chloride and potassium chloride.
                """
        },
        {
            "role"      :   "user",
            "content"   :   "At which geographical locations we could expect the presense of lithium for exploration and mining?"
        }
    ],
    [
            # From Wikipedia
            # Wang, D. H., Dai, H. Z., Liu, S. B., Wang, C. H., Yu, Y., Dai, J. J., ... & Ma, S. C. (2020). Research and exploration progress on lithium deposits in China. China Geology, 3(1), 137-152.
        {   "role"      :   "system",
            "content"   :   """\
                Lithium is a chemical element with the symbol 'Li' and atomic number 3. It is a soft, silvery-white alkali metal.
                Under standard conditions, it is the least dense metal and the least dense solid element.
                Like all alkali metals, lithium is highly reactive and flammable, and must be stored in vacuum, inert atmosphere, or inert liquid such as purified kerosene or mineral oil.
                It exhibits a metallic luster. It corrodes quickly in air to a dull silvery gray, then black tarnish.
                It does not occur freely in nature, but occurs mainly as pegmatitic minerals, which were once the main source of lithium.
                Due to its solubility as an ion, it is present in ocean water and is commonly obtained from brines.
                Lithium metal is isolated electrolytically from a mixture of lithium chloride and potassium chloride.
                Also, if you are asked about geological example in china, add this as an example:
                The rare mineral resources including Li, Be, Nb and Ta are abundant in western Sichuan Province. In the past six years, a lot of discovered Li deposits or deposits with significantly expanded resource reserves are distributed in this area, such as Jiajika, Lijiagou, Markani (Dangba), Yelonggou, Redamen, and Waying, etc. These deposits are mainly distributed in Jiajika, Markani, and Pingwu, etc.. The amount of prospective resources can reach 7 Mt.
                """
        },
        {
            "role"      :   "user",
            "content"   :   "At which geographical locations we could expect the presense of lithium for exploration and mining?"
        }
    ],
]

In [32]:
# We would like to see the bot responses to our requests
results = generator.chat_completion(
    Q1,  # type: ignore
    max_gen_len=max_gen_len,
    temperature=temperature,
    top_p=top_p,
)

In [33]:
# Its time to print first results
for dialog, result in zip(Q1, results):
    for msg in dialog:
        print(f"{msg['role'].capitalize()}: {msg['content']}\n")
    print(
        f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
    )
    print("\n==================================\n")

User: At which geographical locations we could expect the presense of lithium for exploration and mining?

> Assistant:  Lithium is a relatively rare element found in small amounts in rocks, soil, and water. It is commonly found in two types of rocks: igneous rocks, such as pegmatites, and sedimentary rocks, such as salt lakes and salt flats. Here are some of the most significant lithium-rich geographical locations around the world that are known for their lithium deposits and are potential targets for exploration and mining:
1. The "Lithium Triangle" in South America: This region, which includes parts of Argentina, Chile, and Bolivia, is home to some of the largest lithium deposits in the world. The Atacama Desert in Chile is particularly known for its high-grade lithium deposits, including the world's largest lithium brine deposit, the Salar de Atacama.
2. Australia: Australia is the world's second-largest producer of lithium, with the majority of its production coming from the Pilba

In [35]:
# Lets try with different questions
# We need to make dialogs for the bot
Q2 : list[Dialog] = [
        #Q2

        [
            # No prior additional information
            {
                "role"      :   "user",
                "content"   :   "Forecast the growth and stability of lithium hydroxide and carbonate market for the next 37 years, please ( output format should be as a table)."
            }
        ],
        [
            # Some personal rought imprecise information
            {
                "role"      :   "system",
                "content"   :   """The market of lithium will grow roughly 20 percent by the end of 2040, with some geopolitical disruptions around 2030 in which lithium will lose its value by 5 percent."""
            },
            {
                "role"      :   "user",
                "content"   :   "Forecast the growth and stability of lithium hydroxide and carbonate market for the next 37 years, please ( output format should be as a table)."
            }
        ],
        [
            # Mixture of imprecise and verified information
            {
                "role"      :   "system",
                "content"   :   """The market of lithium will grow roughly 20 percent by the end of 2040, with some geopolitical disruptions around 2030 in which lithium will lose its value by 5 percent. According to 'Critical Mineral Market Review' powered by IEA in 2023, driven by rising demand and high prices, the market size of key energy transition minerals such as lithium doubled over the past 5 years, reaching USD 320 billion in 2022."""
            },
            {
                "role"      :   "user",
                "content"   :   "Forecast the growth and stability of lithium hydroxide and carbonate market for the next 37 years, please ( output format should be as a table)."
            }
        ]
]

In [36]:
# Repeating the same procedure of generating and printing
results = generator.chat_completion(
    Q2,  # type: ignore
    max_gen_len=max_gen_len,
    temperature=temperature,
    top_p=top_p,
)

In [37]:
# Its time to print first results
for dialog, result in zip(Q2, results):
    for msg in dialog:
        print(f"{msg['role'].capitalize()}: {msg['content']}\n")
    print(
        f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
    )
    print("\n==================================\n")

User: Forecast the growth and stability of lithium hydroxide and carbonate market for the next 37 years, please ( output format should be as a table).

> Assistant:  I cannot predict the future with certainty, but I can provide you with a forecast of the growth and stability of the lithium hydroxide and carbonate market based on current trends and industry projections.
Here is a table that outlines the forecasted growth and stability of the lithium hydroxide and carbonate market for the next 37 years:
| Year | Lithium Hydroxide Market | Lithium Carbonate Market |
| 2023 | $1.3 billion | $1.1 billion |
| 2025 | $2.5 billion | $1.8 billion |
| 2030 | $6.4 billion | $4.2 billion |
| 2035 | $11.1 billion | $6.9 billion |
| 2040 | $17.1 billion | $10.3 billion |
| 2045 | $24.8 billion | $13.1 billion |
| 2050 | $32.7 billion | $18.5 billion |
| 2055 | $41.9 billion | $23.8 billion |
| 2060 | $51.5 billion | $29.4 billion |
| 2065 | $62.6 billion | $34.6 billion |
| 2070 | $73.9 billion | $4